Buy vs Rent
===

In this exercise, we will be comparing the present value and cash flows of each option of living in a new condo: buying or renting

Here is the link of the GSheet: https://docs.google.com/spreadsheets/d/1lL0jxCZGP51TP5G_CfIzHkjqYz66PFG_HocBAUp1d3o/edit#gid=0

The following links show the interest rates of each bank

กรุงไทย https://krungthai.com/th/personal/loan/housing-loan/16?fbclid=IwAR29mUnFI7fu_jljRXtcgRlZ8JboFtvnC0lw-zcHlrUfro5Kl4JIqzGtyzg

First is to compare rent and mortgage present values and monthly cash flows. For simplicity, we assume monthly compounding interest and moving out in 4 years, maturity is 30 years

We first extract all the historical market price data of each square meter of each condo

In [2]:
from csv import reader
import pandas as pd
import numpy as np
opened_file=open('condo price changes.csv')
read_file=reader(opened_file)
hist=list(read_file)
hist=pd.DataFrame(data=hist[1:],columns=["condo","date","price"])

Here are some of the assumptions that we have

In [3]:
rent={'Abstracts Phahonyothin Park':28000,'Seed Terre':15000,'Central Ratchayothin Park':9000}
offer_price={'Abstracts Phahonyothin Park':5300000,'Seed Terre':5000000,'Central Ratchayothin Park':3300000}
sqm={'Abstracts Phahonyothin Park':47,'Seed Terre':51,'Central Ratchayothin Park':68}
juristic={'Abstracts Phahonyothin Park':10000,'Seed Terre':10000,'Central Ratchayothin Park':10000}
t=4 #number of years before we move out of the condo
T=30 #years maturity of the mortgage
tax=.13 #taxation on top of the monthly payment
mrr=.0712
rT=[[mrr-0.0562,1],[mrr-0.0325,2],[mrr-0.023,3],[mrr-0.01,30]] #as from กรุงไทย 
simple_rt=[[.03,4]]
simple_rt2=[[.01,1],[.99,2]]
#the first field is interest rate from the bank, the second field is the year number until the rate is effective
pv_r=.03 #rate used for present value

In [4]:
mrr=0.06245
krung_thai_2nd_15=[[mrr-0.0562,1],[mrr-0.0325,2],[mrr-0.023,3],[mrr-0.01,30]] #insurance for 15 years
krung_thai_2nd_10=[[mrr-0.0537,1],[mrr-0.0325,2],[mrr-0.02305,3],[mrr-0.01,30]] #insurance for 10 years
krung_thai_2nd_0=[[mrr-0.0512,1],[mrr-0.0325,2],[mrr-0.03,3],[mrr-0.01,30]] #no insurance


Computation of the present value if we rent

In [5]:
def rent_pv(condo,pv_r,s):
    pymt=rent[condo]
    if pv_r==0:
        pv=pymt*s*12
    else:
        a=1/(1+pv_r/12)
        pv=pymt*a*(1-a**(12*s))/(1-a)
    return pv
#already tested

In [6]:
def mortgage_schedule(condo,rt,p):
    b_0=offer_price[condo]
    tminus1=0
    ib_list=[] #list of interest and balance
    for i in rt:
        for k in range(tminus1*12,12*i[1]):
            interest=i[0]*b_0/12
            b_0+=interest-p
            ib_list.append([interest,b_0])
            tminus1=i[1]
    return ib_list
#tested

def mortgage_pymt(condo,rt): #using secant method
    x_2=15000
    x_1=24000
    while abs(mortgage_schedule(condo,rt,x_1)[-1:][0][1])>.01:
        f_2=mortgage_schedule(condo,rt,x_2)[-1:][0][1]
        f_1=mortgage_schedule(condo,rt,x_1)[-1:][0][1]
        x = (x_2 * f_1 - x_1 * f_2) / (f_1 - f_2)
        x_2 = x_1
        x_1 = x
    return x
#tested

def juristic_pv(condo,pv_r,t=t):
    j=juristic[condo] #juristic annual payment
    a=(1+pv_r/12)
    return j*a*(1-a**(12*t))/(1-a**12)

def marketprice(condo,s):
    current_marketprice=float(hist[hist['condo']==condo].iloc[-1,:]['price'])*sqm[condo]
    st=hist[hist['condo']==condo]
    ln_rate=np.log(st['price'].astype(float)).diff(periods=4*s)
    return current_marketprice*np.exp(ln_rate.dropna().sort_values())

In [15]:
def mortgage_schedule(condo,rt,dp,p): #p=monthly fixed payment, #dp=downpayment as %
    b_0=offer_price[condo]*(1-dp)
    tminus1=0
    ib_list=[] #list of interest and balance
    for i in rt:
        for k in range(tminus1*12,12*i[1]):
            interest=i[0]*b_0/12
            b_0+=interest-p
            ib_list.append([interest,b_0])
            tminus1=i[1]
    return ib_list
#tested

def mortgage_pymt(condo,rt,dp): #using secant method
    x_2=15000
    x_1=24000
    while abs(mortgage_schedule(condo,rt,dp,x_1)[-1:][0][1])>.01:
        f_2=mortgage_schedule(condo,rt,dp,x_2)[-1:][0][1]
        f_1=mortgage_schedule(condo,rt,dp,x_1)[-1:][0][1]
        x = (x_2 * f_1 - x_1 * f_2) / (f_1 - f_2)
        x_2 = x_1
        x_1 = x
    return x
#tested

def juristic_pv(condo,pv_r,t=t):
    j=juristic[condo] #juristic annual payment
    a=(1+pv_r/12)
    return j*a*(1-a**(12*t))/(1-a**12)

def marketprice(condo,s):
    current_marketprice=float(hist[hist['condo']==condo].iloc[-1,:]['price'])*sqm[condo]
    st=hist[hist['condo']==condo]
    ln_rate=np.log(st['price'].astype(float)).diff(periods=4*s)
    return current_marketprice*np.exp(ln_rate.dropna().sort_values())

In [31]:
def mortgage_pv(condo,pv_r,rt,dp,s):
    #marketprice - (mortgage payments + mortgage balance + juristic)
    pymt=mortgage_pymt(condo,rt,dp)
    pv_a=1/(1+pv_r/12)
    pymt_pv=pymt * pv_a * (1 - pv_a ** (12 * s)) / (1 - pv_a)
    
    sched=mortgage_schedule(condo,rt,dp,pymt)
    pv_b=sched[12*t+1][0][1] * pv_a ** (12 * s)
    
    cost=pymt_pv + pv_b + juristic_pv(condo,pv_r,s)
    hello=[]
    for k in marketprice(condo):
        hello.append(k/(1+pv_r)**(12*s)-costs)
    return hello

|loan type|interest|
|-|-|
|MLR|6-8.47%|
|MRR|7.12-12.75%|
|MOR|6.87-8.65%|

Occupancy is from 65-90%; annual increase in value 8%

now we compute for the future marketprice

In [46]:
def summary(condo,pv_r,rt,dp,t):
    a=1/(1+pv_r/12)
    pymt=mortgage_pymt(condo,rt,dp)
    mortgage_pymt_pv=pymt*a*(1-a**(12*t))/(1-a)
    mortgage_remaining_balance=mortgage_schedule(condo,rt,dp,pymt)[12*t+1][1]
    print("The current rent price is ",'{:,.2f}'.format(rent[condo]))
    print("The current marketprice is ",'{:,.2f}'.format(float(hist[hist['condo']==condo].iloc[-1,:]['price'])*sqm[condo]))
    print("The offer price is ",'{:,.2f}'.format(offer_price[condo]))
    print("The monthly mortgage payment is ",'{:,.2f}'.format(pymt))
    print("The annual juristic is ",'{:,.2f}'.format(juristic[condo]))
    print("After ", t ," years, the remaining mortgage balance is ",'{:,.2f}'.format(mortgage_schedule(condo,rt,dp,pymt)[12*t][1]))
    print("After ",t," years, the marketprice of the condo is ",'{:,.2f}'.format(marketprice(condo,t).median()))
    print("After ",t," years, the marketprice of the condo worst case is ",'{:,.2f}'.format(marketprice(condo,t).iloc[0]*a**(12*t)))
    print("Present value of the median :" , '{:,.2f}'.format(marketprice(condo,t).median()*a**(12*t) - offer_price[condo]*dp - mortgage_pymt_pv - juristic_pv(condo,pv_r) - mortgage_remaining_balance*a**(12*t)))
    print("Present value of the worst case :" , '{:,.2f}'.format(marketprice(condo,t).iloc[0]*a**(12*t) - mortgage_pymt_pv - juristic_pv(condo,pv_r) - mortgage_remaining_balance*a**(12*t)))
    print("The present value of all the rent payments is ",'{:,.2f}'.format(-rent_pv(condo,pv_r,t)))

In [56]:
summary('Abstracts Phahonyothin Park',.07,krung_thai_2nd_0,0,4)

The current rent price is  28,000.00
The current marketprice is  5,034,029.00
The offer price is  5,300,000.00
The monthly mortgage payment is  27,076.06
The annual juristic is  10,000.00
After  4  years, the remaining mortgage balance is  4,598,987.41
After  4  years, the marketprice of the condo is  6,076,354.50
After  4  years, the marketprice of the condo worst case is  3,188,732.26
Present value of the median : -52,757.60
Present value of the worst case : -1,460,172.74
The present value of all the rent payments is  -1,169,285.64


In [53]:
summary('Abstracts Phahonyothin Park',.07,krung_thai_2nd_0,0.1,4)

The current rent price is  28,000.00
The current marketprice is  5,034,029.00
The offer price is  5,300,000.00
The monthly mortgage payment is  24,368.46
The annual juristic is  10,000.00
After  4  years, the remaining mortgage balance is  4,139,088.67
After  4  years, the marketprice of the condo is  6,076,354.50
After  4  years, the marketprice of the condo worst case is  3,188,732.26
Present value of the median : -122,348.11
Present value of the worst case : -999,763.25
The present value of all the rent payments is  -1,169,285.64


In [54]:
summary('Seed Terre',.07,krung_thai_2nd_0,0.1,4)

The current rent price is  15,000.00
The current marketprice is  5,285,385.00
The offer price is  5,000,000.00
The monthly mortgage payment is  22,989.11
The annual juristic is  10,000.00
After  4  years, the remaining mortgage balance is  3,904,800.63
After  4  years, the marketprice of the condo is  5,796,812.17
After  4  years, the marketprice of the condo worst case is  3,240,184.91
Present value of the median : -69,245.37
Present value of the worst case : -713,762.37
The present value of all the rent payments is  -626,403.02


In [55]:
summary('Central Ratchayothin Park',.07,krung_thai_2nd_0,0.1,4)

The current rent price is  9,000.00
The current marketprice is  3,789,980.00
The offer price is  3,300,000.00
The monthly mortgage payment is  15,172.81
The annual juristic is  10,000.00
After  4  years, the remaining mortgage balance is  2,577,168.42
After  4  years, the marketprice of the condo is  4,226,162.92
After  4  years, the marketprice of the condo worst case is  2,133,777.60
Present value of the median : 241,824.03
Present value of the worst case : -491,063.03
The present value of all the rent payments is  -375,841.81
